In [1]:
!pip install openai
import openai
import base64
from bs4 import BeautifulSoup
import requests
from google.colab import userdata
import json

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [3]:
with open('scrapedTextNoImages.txt', 'r') as file:
  text = file.read()

In [4]:
pages = text.split("\u3019")
# Removes the initial unicode character used to denote the start of a page
pages = [x[1:] for x in pages]

In [10]:
def questionGeneration(page):
    backgroundInfo = '''
    I am training a large language model (LLM) using information from a wiki.
    My goal is to evaluate the model's comprehension, factual recall, and reasoning
    abilities by generating a set of questions to test it. Please help me come up
    with a comprehensive list of questions that quizzes the model on everything
    on the page but can also effectively assess the model's
    accuracy in these areas:

    Comprehension: Test the model's ability to understand and interpret the
    information presented. The questions should measure how well the model
    grasps the meaning, context, and relationships within the content.

    Factual Recall: Test the model’s ability to accurately recall facts,
    figures, and specific details from the information. The questions
    should challenge the model’s ability to retrieve precise, factual
    information without distortion.

    Reasoning: Test the model's capacity for logical inference,
    problem-solving, and drawing conclusions based on the information provided.
    The questions should assess how well the model applies reasoning to answer
    questions that require more than just recalling information.

    Please avoid vague descriptions and focus on specific details,
    ensuring the description accurately reflects the visual content. Also avoid
    any hallucinations.

    I want you to return a object in JSON with the following keys:
    1. Keyname: "url". Contains a URL that is found at the beginning of the text that I provide for you.
    2. Keyname: "Q/A". A Q/A array where each array element is another object in JSON notation
    containing a question key that has a question and then an answer key containing the answer.
    3. A number key that lists the total number of questions

    Make sure to use double quotes everywhere, unless you're writing a quote
    within a double quote in which case you can use single quotes
    '''
    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {userdata.get('OPEN_API_KEY')}"
    }

    payload = {
      "model": "gpt-4o-mini",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": f"{backgroundInfo}. Now here is the text for the wiki page: {page}"
            },
          ]
        }
      ],
      "max_tokens": 3000
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # Prints the response, then reformat's GPTs json string
    print(response.json()['choices'][0]['message']['content'])
    return response.json()['choices'][0]['message']['content'].replace("\n", "")[7:-3]

In [24]:
for i in range(334, len(pages)):
  print(f"Generating questions about page {i}")
  with open('questionsList.json', 'a') as file:
    json.dump(json.loads(questionGeneration(pages[i])), file, ensure_ascii=False, indent=4)
with open('questionsList.json', 'a') as file:
  file.write("]")

Generating questions about page 334
Please provide the text from the wiki page that you would like me to use to generate the questions.


JSONDecodeError: Expecting value: line 1 column 1 (char 0)